1)Написать приложение, которое собирает основные новости с сайтов mail.ru, lenta.ru, yandex.news
Для парсинга использовать xpath. Структура данных должна содержать:
* название источника,
* наименование новости,
* ссылку на новость,
* дата публикации

2)Сложить все новости в БД

In [1]:
import requests
from lxml import html
import pandas as pd
from pprint import pprint
import re
from datetime import datetime as dt
from pymongo import MongoClient

In [2]:
class News_parsing():
    
      
    def get_news_info(self):
        
        self.main_page = requests.get(self.main_link, headers = self.headers_user_agent)
        
        links, dates, titles, sources = self._news_info(self.main_page, self.news_links_xpath)
        
        info_news = pd.DataFrame({'link':links, 'date':dates, 'title':titles, 'source':sources})
        
        return info_news
        
        
    def _news_link(self, main_page, news_links_xpath):
        news_href = html.fromstring(main_page.text).xpath(self.news_links_xpath)
        
        return news_href
        
        
    def _news_info(self, main_page, news_links_xpath):
        
        links = self._news_link(self.main_page, self.news_links_xpath)
        
        dates = []
        titles = []
        sources = []
        
        
        for link in links:
            page = html.fromstring(requests.get(link, headers = self.headers_user_agent).text)
            
            date = page.xpath(self.news_date_xpath)
            title = page.xpath(self.news_title_xpath)
            
            source = self.news_source_xpath
            if source[0] == '/':
                source = page.xpath(self.news_source_xpath)
                sources.append(source[0])
            else:
                sources.append(source)
            date = self._date_formetter(date)
            title = [''.join(title).replace('\xa0', ' ')]

            dates.append(date[0])
            titles.append(title[0])
        
        return links, dates, titles, sources
    
    
    def _date_formetter(self, date):
        date = ''.join(list(map(lambda x: x.split('T')[0], date))).split('-')[::-1]
        date = ['.'.join(date)]
        return date
            

In [3]:
class Mail_news(News_parsing):
    def __init__(self):
        self.headers_user_agent = {'User-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36'}
    
        self.main_link = 'https://mail.ru/'
        
        self.news_links_xpath = "//div[@class='news-item i-fade-white i-fade-full i-nowrap news-item_inline']//a[not(@class='news-item__label i-color-black i-inline')]/@href"
        
        self.news_date_xpath = "//span[@class='note__text breadcrumbs__text js-ago']/@datetime | //div[@class='breadcrumbs breadcrumbs_article']/span[1]/@datetime"
        self.news_title_xpath = "//div[@class='hdr__wrapper']/span[@class='hdr__text']/h1[@class='hdr__inner']/text() | //div[@class='p-intro__caption']/h1[@class='p-intro__title']/text()"
        self.news_source_xpath = "//a[@class='link color_gray breadcrumbs__link']/span/text()"

In [4]:
class Lenta_news(News_parsing):
    def __init__(self):
        self.headers_user_agent = {'User-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36'}
    
        self.main_link = 'https://lenta.ru'
        
        self.news_links_xpath = "//div[contains(@class, 'js-main__content') and contains(@class, 'span8')]//div[contains(@class, 'span4')]//time[@class='g-time']/parent::*/@href"
        self.news_links_xpath += " | "
        self.news_links_xpath += "//div[contains(@class, 'js-content') and contains(@class, 'row')][1]//div[contains(@class, 'span4')]//a[contains(@class, 'js-dh')]/@href"
        self.news_links_xpath += " | "
        self.news_links_xpath += "//div[contains(@class, 'js-content') and contains(@class, 'row')][2]//div[contains(@class, 'span4')]//div[contains(@class, 'item') and contains(@class, 'news')]//div[contains(@class, 'title')]//a/@href"
        
        self.news_date_xpath = "//div[contains(@class, 'js-topic__header')]//div[contains(@class, 'b-topic__info')]//time[@class='g-date']/@datetime"
        self.news_title_xpath = "//div[contains(@class, 'b-topic__header')]//h1[contains(@class, 'b-topic__title')]/text() | //div[contains(@class, 'premial-header__title')]/text()"
        
        self.news_source_xpath = "Lenta.ru"
        
        
    def _news_link(self, main_page, news_links_xpath):
        news_href = html.fromstring(main_page.text).xpath(self.news_links_xpath)
        
        def __temp_link(x):
            if re.search(r'^(\/news).+', x): 
                x = self.main_link + x
            else:
                x = None
            return x
        
        
        return list(filter(None, map(lambda x: __temp_link(x), news_href)))
        

In [5]:
class Yandex_news(News_parsing):
    def __init__(self):
        self.headers_user_agent = {'User-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36'}
    
        self.main_link = 'https://yandex.ru/news/'
        
        self.news_links_xpath = "//div[@class='stories-set__main-item']//h2[@class='story__title']//a/@href"
        self.news_links_xpath += " | "
        self.news_links_xpath += "//table[@class='stories-set__items']//td[@class='stories-set__item']//h2[@class='story__title']//a/@href"
       
        self.news_date_xpath = "//div[@class='stories-set__main-item']//div[@class='story__date']/text()"
        self.news_date_xpath += " | "
        self.news_date_xpath += "//table[@class='stories-set__items']//td[@class='stories-set__item']//div[@class='story__date']/text()"
        
        
        self.news_title_xpath = "//div[@class='stories-set__main-item']//h2[@class='story__title']//a/text()"
        self.news_title_xpath += " | "
        self.news_title_xpath += "//table[@class='stories-set__items']//td[@class='stories-set__item']//h2[@class='story__title']//a/text()"
        
        
        self.news_source_xpath = self.news_date_xpath
        
        
    def _news_link(self, main_page, news_links_xpath):
        news_href = html.fromstring(main_page.text).xpath(news_links_xpath)
        
        def __temp_link(x):
            if re.search(r'^(\/).+', x): 
                x = 'https://yandex.ru' + x
                
            return x
        
        
        return list(filter(None, map(lambda x: __temp_link(x), news_href)))
        
        
    def _get_date_title(self, main_page, link_xpath):
        date_or_title = html.fromstring(main_page.text).xpath(link_xpath)
        
                
        return date_or_title
    
    def _news_info(self, main_page, news_links_xpath):
        
        links = self._news_link(self.main_page, self.news_links_xpath)
        
        
        months={'января':'01', 
                'февраля':'02', 
                'марта':'03', 
                'апреля':'04', 
                'мая':'05', 
                'июня':'06', 
                'июля':'07', 
                'августа':'08', 
                'сентября':'09', 
                'октября':'10', 
                'ноября':'11', 
                'декабря':'12'}
        yesterday='вчера'
        postyesterday='позавчера'
        
        def __temp_date(date):
            if '\xa0' not in date:
                date = list(dt.today().timetuple())[0:3]
                date[1] = str(date[1]).rjust(2, '0')
                date = '.'.join(map(str,date[::-1]))
            else:
                date = ' '.join(date.split('\xa0')).split(' ')
                for i, s in enumerate(date):
                    if s in months.keys():
                        d = date[i-1]
                        m = months[s]
                        y = list(dt.today().timetuple())[0]
                        date='.'.join([d,m,y])
                    elif s == yesterday:
                        date=list(dt.fromordinal(dt.toordinal(dt.today())-1).timetuple())[0:3]
                        date[1] = str(date[1]).rjust(2, '0')
                        date = '.'.join(map(str,date[::-1]))
                    elif s == postyesterday:
                        date=list(dt.fromordinal(dt.toordinal(dt.today())-2).timetuple())[0:3]
                        date[1] = str(date[1]).rjust(2, '0')
                        date = '.'.join(map(str,date[::-1]))
                        
            return date
        
        
        def __temp_source(date):
            
            date = ' '.join(date.split('\xa0')).split(' ')
            
            if '\xa0' not in date and yesterday not in date and postyesterday not in date:
                source = ' '.join(map(str,date[:-1:]))
            else:
                for i, s in enumerate(date):
                    if s in months.keys():
                        source = ' '.join(map(str,date[:i-1:]))
                    elif s == yesterday or s == postyesterday:
                        source = ' '.join(map(str,date[:i:]))
                    
                        
            return source
                
        dates = list(map(lambda x: __temp_date(x), self._get_date_title(self.main_page, self.news_date_xpath)))
        titles = self._get_date_title(self.main_page, self.news_title_xpath)
        sources = list(map(lambda x: __temp_source(x), self._get_date_title(self.main_page, self.news_source_xpath)))
        

        return links, dates, titles, sources
    
    
        

In [6]:
news_mail = Mail_news()
info_mail = news_mail.get_news_info()

news_lenta = Lenta_news()
info_lenta = news_lenta.get_news_info()

news_yandex = Yandex_news()
info_yandex = news_yandex.get_news_info()

In [7]:
news = info_mail.copy()
news = news.merge(info_lenta, how='outer')
news = news.merge(info_yandex, how='outer')
news_for_db = news.to_dict('records')

In [8]:
client = MongoClient('localhost', 27017)
parsing_news = client['parsing_news']

In [9]:
for new in news_for_db:
    try:
        parsing_news.news.update_one({'link':new['link']}, {'$set':new}, upsert=True)
    except:
        pass

In [10]:
parsing_news.news.count_documents({})

156